# Wordlift Vector Store

In this notebook we are going to show how to use Wordlift as a Vector store to be used in LlamaIndex.

### Setting up environments

Install Llamaindex and Wordlift vector store using pip 

In [ ]:
%pip install llama-index
%pip install llama-index-vector-stores-wordlift
%pip install nest_asyncio

In [ ]:
import sys
import os


sys.path.append(
    os.path.abspath(
        "/Users/ideapad/Desktop/wordlift/llama_index/llama-index-integrations/vector_stores/llama-index-vector-stores-wordlift"
    )
)

In [ ]:
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.wordlift import WordliftVectorStore
from llama_index.core.embeddings.utils import get_cache_dir
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import Document
import nest_asyncio

nest_asyncio.apply()

Setup OpenAI API

In [ ]:
import os
import openai

# openai.api_key = os.environ[""]

In [ ]:
cache_folder = os.path.join(get_cache_dir(), "models")
os.makedirs(cache_folder, exist_ok=True)


model_name = "nomic-ai/nomic-embed-text-v1"
embed_model = HuggingFaceEmbedding(
    model_name=model_name, cache_folder=cache_folder, trust_remote_code=True
)
Settings.embed_model = embed_model

Download and prepare the sample dataset

In [ ]:
!mkdir 'data\paul_graham\'
!curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [ ]:
documents = SimpleDirectoryReader("./data/paul_graham").load_data()

documents[0].metadata[
    "entity_id"
] = "https://data.wordlift.io/wl1505016/paul_graham_essay-txt"

### Create Wordlift Vectore Store

To create a Wordlift vector store instance you just need the key of your Wordlift Knowledge Graph. Remember that there is a key for each Knowledge Graph

In [ ]:
vector_store = WordliftVectorStore.create("your_key")

# set Wordlift vector store instance as the vector store
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

In [ ]:
# test the vector store query
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)